Importovanie kniznic

In [ ]:

import pandas as pd
import numpy as np
import openai
import google.generativeai as genai
from chatbot.gemini_embdeddings import gemini_embdeddings_create
from chatbot.gpt_embdeddings import openai_embedding_create
import random
import textwrap
import re 

from nltk.translate.bleu_score import sentence_bleu
from nltk.translate import meteor_score

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rouge_score import rouge_scorer
from nltk.tokenize import word_tokenize

from sklearn.metrics.pairwise import cosine_similarity



In [ ]:

#Funkcia na vypocet bert score , pomocou kniznice BERTscorer
def bert_score(answer,expected_answer):
    from bert_score import BERTScorer
    scorer = BERTScorer(lang="sk")
    P1, R1, F1_1 = scorer.score([answer], [expected_answer])
    return F1_1.tolist()[0]


#Táto funkcia používa embdeddings modely  na premenu textu na číselné reprezentacie slov a potom vypočíta kosínusovú podobnosť medzi týmito embeddings, aby zistila, ako podobné sú dva vstupy.
def create_embdeddings_similarity(otazka1,otazka2):
    # Vytvorí embeddings pre  otázky pomocou modelu 'text-embedding-3-large'
    response1 = openai.Embedding.create(
            model="text-embedding-3-large",  
            input=otazka1
        )
    # Konvertuje odpoveď na pole numpy
    embedding1=np.array(response1['data'][0]['embedding'])
    

    response2 = openai.Embedding.create(
            model="text-embedding-3-large",  
            input=otazka2
        )
    embedding2=np.array(response2['data'][0]['embedding'])
    # Preusporiada embeddings na formát potrebný pre výpočet podobnosti
    embedding1 = embedding1.reshape(1, -1)
    embedding2 = embedding2.reshape(1, -1)

    # Vypočíta kosínusovú podobnosť medzi dvoma embeddings
    similarity_score = cosine_similarity(embedding1, embedding2)
    return similarity_score[0][0]


def count_words(text):
    words = re.findall(r'\w+', text)  # Use regular expression to find all words
    return len(words), words 


#Táto funkcia vypočíta podobnosť dvoch textov tak, že ich prevedie na vektory TF-IDF a potom vypočíta kosínusovú podobnosť medzi týmito vektormi. 
def custom_similarity_metric(ocakavana_odpoved, vytvorena_odpoved):
    # Vytvorenie TF-IDF vektorizátorov pre očakávanú a vytvorenú odpoveď
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([ocakavana_odpoved, vytvorena_odpoved])

    # Výpočet kosínusovej podobnosti medzi vektormi TF-IDF
    similarity_matrix = cosine_similarity(tfidf_matrix)

    # Kosínusová podobnosť pre konkrétny pár odpovedí
    custom_similarity_score = similarity_matrix[0, 1]

    return custom_similarity_score

def calculate_similarity(embedding1, embedding2):
    embedding1 = np.array(embedding1, dtype=np.float32)
    embedding2 = np.array(embedding2, dtype=np.float32)

    # Calculate cosine similarity between two embeddings
    dot_product = np.dot(embedding1, embedding2)
    norm1 = np.linalg.norm(embedding1)
    norm2 = np.linalg.norm(embedding2)

    # Check for zero division to avoid runtime warnings
    if norm1 == 0 or norm2 == 0:
        return 0.0
    
    similarity = dot_product / (norm1 * norm2)
    return similarity

#funckia na najdenie najrlevantnejsieho dotazu v datasete pre model Gemini
def find_best_passage(query, dataframe):
        model='models/embedding-001'
        """
        Compute the distances between the query and each document in the dataframe
        using the dot product.
        """
        query_embedding = genai.embed_content(model=model,
                                                content=query,
                                                task_type="retrieval_query")
        dot_products = np.dot(np.stack(dataframe['Embeddings']), query_embedding["embedding"])
        idx = np.argmax(dot_products)
        return dataframe.iloc[idx]['answers'] # Return text from index with max value


#funkcia na vytovrenie "promptu" pre model gemini ktora urucje insturkcie pre model,otazku od pouzivatela a najdeny text ktory sa pouzije ako kontext
def make_prompt(query, relevant_passage):
  escaped = relevant_passage.replace("'", "").replace('"', "").replace("\n", " ")
  prompt = textwrap.dedent("""si pomocnik ktory pomaha odpovedat studentom ohladom studia.Informacie ziskaj z passage ktore je pridane 
  QUESTION: '{query}'
  PASSAGE: '{relevant_passage}'

    ANSWER:
  """).format(query=query, relevant_passage=escaped)

  return prompt

#funkcia pre model GPT , ktora v prvej casti ulohy otazku pretransformuje na embdeddings a nasledne podla kosinusovje podobnosti hlada najrelevantnejsiu zhodu
#po najdeni zhody sa vytvori zakladny prompt ktory sa posiela pomocou API na vygeneoravnie odpovede
def custombased(text,temperature,gpt_model,top_p):
    response = openai.Embedding.create(
            model="text-embedding-3-large",  
            input=text
        )
    question_embedding=response['data'][0]['embedding']


    best_match = None
    best_similarity = 0.0

    for index, row in openai_embd.iterrows():
            entry_embedding = row['embedding']   
        
            similarity = calculate_similarity(question_embedding, entry_embedding)


            if similarity > best_similarity:
                best_similarity = similarity
                best_match=row['answers']
                
    #print(best_match)
                

    query = f"""Pouzi dole priradeny clanok k odpovedi na otazku , ak nevies odpoved odpoved "Nemam dostatok informaciu na tuto otazku"  "

            clanok:
            \"\"\"
            {best_match}
            \"\"\"

            Otazka: {text}"""
    #print(query)
    #conversation = [
    #        {"role": "system", "content": "Si Skolsky asistent na techncikej univerzite v kosiciach a mas pomoct studentom odpoved na otazky"},
    #        {"role": "user", "content": f"Question: {query}"}
    #    ]


    conversation = [
            {"role": "system", "content": "Si skolsky asistent ktory odpoveda studentom na otazky vygeneruj odpoved maximalne v dlzke 256 tokenov "},
            {"role": "user", "content": f"Question: {text}"},
            {"role": "assistant", "content": f"Context: Pouzi dole priradeny clanok k odpovedi na otazku  clanok:{best_match}"}
        ]

    response = openai.ChatCompletion.create(
    model=gpt_model,
    messages=conversation,
    max_tokens=500,
    temperature=temperature,
    top_p=top_p
    
    
    )


    answer = response['choices'][0]['message']['content'].strip()
    return answer



#funkcia ktora spravi vsetky potrebne kroky pre vygenerovanie dotazku pre Gemini
def gemini_resp(query,temperature,top_p):
    #mesure time of execution 
    generation_config = {
    "temperature": temperature,
    "top_p": top_p
    }
    passage =find_best_passage(query, gemini_embd)
    #print(passage)
    prompt = make_prompt(query, passage)
    model = genai.GenerativeModel('models/gemini-pro')
    answer = model.generate_content(prompt,generation_config=generation_config)
    return answer.text

Nacitanie dat a nastavenie dolezitych casti

In [ ]:
openai.api_key = '#################################'
genai.configure(api_key='#################################')

Nacitanie Dat a transofmracie embdeddings


In [ ]:
print("--------------------------------CREATING GEMINI EMBEDDEDINGS--------------------------------")
#gemini_embd=gemini_embdeddings_create()
gemini_embd=pd.read_csv('gemini_embdeddings.csv')
gemini_embd['Embeddings'] = gemini_embd.Embeddings.apply(eval).apply(np.array)
print("--------------------------------CREATING OPENAI EMBEDDEDINGS--------------------------------")
#openai_embd=openai_embedding_create()
openai_embd=pd.read_csv('openai_embdeddings.csv')
openai_embd['embedding'] = openai_embd.embedding.apply(eval).apply(np.array)
print("--------------------------------CREATING EMBEDDEDINGS FINISHED--------------------------------")

1.Vlastnost  (Robustness to Noise)

Funkcia test_robustness testuje robustnosť AI modelu tým, že simuluje, ako model reaguje na rôzne úrovne "šumu" alebo chýb v textových vstupoch. Šum môže zahŕňať preklepove chyby slová, čo je situácia, ktora sa môžu vyskytnúť pri skutočnom použití. Poďme prejsť kód krok za krokom:

Definícia Funkcie test_robustness:
Funkcia prijíma dva parametre: prompt (textový vstup, na ktorom sa má testovať robustnosť) a noise_levels (zoznam úrovní šumu, ktoré sa majú testovať).
Pre každú úroveň šumu vo zozname noise_levels funkcia vytvára šumivý prompt pomocou funkcie add_noise a následne používa túto upravenú verziu ako vstup pre model.
Generovanie "Noisy Prompt" cez Funkciu add_noise:
    Funkcia add_noise prijíma prompt a noise_level ako vstupy.
    noisy_prompt je konvertovaný na zoznam znakov, aby bolo možné jednoducho meniť jednotlivé znaky.
Pre každý znak v prompt sa rozhoduje, či ho zmeniť alebo nie, závisí od pravdepodobnosti určenej noise_level.


In [ ]:
def test_robustness(prompt, noise_levels):
    for noise_level in noise_levels:
        noisy_prompt = add_noise(prompt, noise_level)
        #print(noisy_prompt)
        response = custombased(noisy_prompt,0,'gpt-4-0125-preview')
        
        print(f"Noisy Prompt: {noisy_prompt}")
        print(f"Response: {response}\n")
        #return response

def add_noise(prompt, noise_level):
    noisy_prompt = list(prompt)
    for i, char in enumerate(noisy_prompt):
        if random.random() < noise_level:
            # Add typographical errors or misspellings
            if char.isalpha():
                noisy_prompt[i] = random.choice([c for c in 'abcdefghijklmnopqrstuvwxyz'])

            # Add word order changes (e.g., shuffling words)
            elif char.isspace() and i > 0 and noisy_prompt[i - 1].isspace():
                j = i + 1
                while j < len(noisy_prompt) and not noisy_prompt[j].isspace():
                    j += 1
                word = noisy_prompt[i:j]
                random.shuffle(word)
                noisy_prompt[i:j] = word

    return ''.join(noisy_prompt)

Test 

In [ ]:
prompt = "Aký je rozsah predmetu objavovanie znalostí v týždennom rozsahu?"
noise_levels = [0.00, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40, 0.45, 0.50] 
test_robustness(prompt, noise_levels)

Variabilnost pri rovnakej otazke ale inej variancie

Funkcia ktora testuje variabilnost otazok a nasledne vypocita bert score a similirty score s ocakvanou odpovedou

In [ ]:
def check_variarity(questions,expected_answer):
    for question in questions:
        response = custombased(question,0,'gpt-4-0125-preview')
        #print(response)
        similirity_score=create_embdeddings_similarity(response,expected_answer)
        bert_scr=bert_score(response,expected_answer)
        print (f"similirity_score :{similirity_score} , bert score: {bert_scr}")

expected_answer='Rozsah predmetu objavovanie znalostí je týždenne 2 hodiny prednášok, 1 hodina laboratórneho cvičenia a 1 hodina projektovej práce.'
questions = [
    "Aký je rozsah predmetu objavovanie znalostí v týždennom rozsahu?",
    "opis predmet objavovanie znalosti",
    "Koľko hodín v týždni je venovaných objavovaniu znalostí?",
    "Aký časový rozsah pokrýva predmet objavovanie znalostí v jednom týždni?",
    "Koľko času treba venovať objavovaniu znalostí počas jedného týždňa?",
    "V akom časovom rozsahu sa venujeme objavovaniu znalostí každý týždeň?",
    "Ako dlho trvá predmet objavovanie znalostí každý týždeň?",
]
check_variarity(questions,expected_answer)

Funkcia na vypocitanie efektu temperatue na generovanu odpoved, vypocet skore funguje rovnako ako vo funkcii check_variarity

In [ ]:
def temperature_effect(question,expected_answer,temperature_list):
    for temperature in temperature_list:
        response = custombased(question,temperature,'gpt-4-turbo-preview',0.5)
        #print(response)
        gpt_word_count, gpt_words = count_words(response)
        
        similirity_score=create_embdeddings_similarity(response,expected_answer)
        bert_scr=bert_score(response,expected_answer)
        print (f"temperature:{temperature} similirity score:{similirity_score} bert score:{bert_scr},gpt_word_count:{gpt_word_count}")
expected_answer='Hlavnou anotáciou predmetu je prehĺbenie znalostí v oblasti objavovania znalostí a dolovania v dátach, proces objavovania znalostí, podrobná náplň jednotlivých fáz tohto procesu podľa metodiky CRISP-DM, metódy dolovania v dátach a ich rozdelenie, prediktívne dolovanie v dátach, metriky pre meranie podobnosti a vzdialenosti a ďalšie súvisiace témy.'
temperature_list=[0,0.1,0.2]
question="Aká je hlavná anotácia predmetu objavovanie znalostí?"
temperature_effect(question,expected_answer,temperature_list)

Funkcia na porovanie modelov, nasledne vygenerovane odpovede sa porovnaju s ocakavanym vystupom a porovna sa ich podobnost a kvalita

In [ ]:
def compare_models(question,expected_answer,temperature,gpt_model):
    for model in gpt_model:
        response = custombased(question,temperature,model)
        #print(response)
        
        similirity_score=create_embdeddings_similarity(response,expected_answer)
        bert_scr=bert_score(response,expected_answer)
        print (f"model name:{model} similirity score:{similirity_score} bert score:{bert_scr}")

expected_answer='Hlavnou anotáciou predmetu je prehĺbenie znalostí v oblasti objavovania znalostí a dolovania v dátach, proces objavovania znalostí, podrobná náplň jednotlivých fáz tohto procesu podľa metodiky CRISP-DM, metódy dolovania v dátach a ich rozdelenie, prediktívne dolovanie v dátach, metriky pre meranie podobnosti a vzdialenosti a ďalšie súvisiace témy.'
question="Aká je hlavná anotácia predmetu objavovanie znalostí?"
gpt_model=['gpt-4-0125-preview','gpt-4-turbo-preview','gpt-4-1106-preview','gpt-4','gpt-4-0613','gpt-3.5-turbo-0125','gpt-3.5-turbo','gpt-3.5-turbo-1106']
compare_models(question,expected_answer,0,gpt_model)

Funkcia na vygenerovanie rozdielnych kombinaici parametrov temperature a top_p, nasledne sa vypocita kvalita a skore odpovede

In [ ]:
expected_answer='Hlavnou anotáciou predmetu je prehĺbenie znalostí v oblasti objavovania znalostí a dolovania v dátach, proces objavovania znalostí, podrobná náplň jednotlivých fáz tohto procesu podľa metodiky CRISP-DM, metódy dolovania v dátach a ich rozdelenie, prediktívne dolovanie v dátach, metriky pre meranie podobnosti a vzdialenosti a ďalšie súvisiace témy.'
question="Aká je hlavná anotácia predmetu objavovanie znalostí?"

def create_questions():
    temperature_list = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0]
    top_p_list = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
    
    # Create an empty list to store dictionaries
    data = []

    for temperature in temperature_list:
        for top_p in top_p_list:
            print(f"tempreture:{temperature}, top_p:{top_p}")
            gpt_response = custombased(question, temperature, 'gpt-4-turbo-preview', top_p)
            gemini_response = gemini_resp(question, temperature, top_p)
            gpt_similarity_score = create_embdeddings_similarity(gpt_response, expected_answer)
            gpt_bert_score = bert_score(gpt_response, expected_answer)
            gemini_similarity_score = create_embdeddings_similarity(gemini_response, expected_answer)
            gemini_bert_score = bert_score(gemini_response, expected_answer)
            
            # Count words and get word list for each answer
            gpt_word_count, gpt_words = count_words(gpt_response)
            gemini_word_count, gemini_words = count_words(gemini_response)
            
            # Append GPT results to the list
            data.append({'model': 'GPT',
                         'temperature': temperature,
                         'top_p': top_p,
                         'model_answer': gpt_response,
                         'expected_answer': expected_answer,
                         'similarity_score': gpt_similarity_score,
                         'bert_score': gpt_bert_score,
                         'word_count': gpt_word_count
                         })
            
            # Append Gemini results to the list
            data.append({'model': 'Gemini',
                         'temperature': temperature,
                         'top_p': top_p,
                         'model_answer': gemini_response,
                         'expected_answer': expected_answer,
                         'similarity_score': gemini_similarity_score,
                         'bert_score': gemini_bert_score,
                         'word_count': gemini_word_count
                         })
    
    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(data)
    
    # Save the DataFrame to a CSV file
    df.to_csv('model_responses.csv', index=False)

# Call the function to create the database
create_questions()